In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



from datetime import datetime
import pause

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import sqlite3
import sqlalchemy

import schedule


from schedule import every, repeat, run_pending
import time


engine = sqlalchemy.create_engine('sqlite:///QQQ_information.db')
connection = sqlite3.connect('QQQ_information.db')
cursor = connection.cursor()

PATH = '/Users/hanshan/Downloads/chromedriver'

ser = Service(PATH)
op = webdriver.ChromeOptions()
op.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

s = webdriver.Chrome( '/Users/hanshan/Downloads/chromedriver',options=op)
s.get('https://client.schwab.com/Areas/Trade/Options/Chains/Index.aspx#symbol/QQQ;strategy/greeks;expiration/custom|2022++06+13&QQQ:25&QQQ;strike/all;expandcollapse/3|2022++06+15&QQQ++06+17&QQQ:04&QQQ:25&QQQ')
url = s.command_executor._url
#'https://client.schwab.com/Areas/Trade/Options/Chains/Index.aspx#symbol/QQQ;strategy/greeks;expiration/custom|2022++02+02&QQQ:25&QQQ;strike/all;expandcollapse/3|2022++01+31&QQQ++02+02&QQQ:04&QQQ:25&QQQ')
url = s.command_executor._url

session_id = s.session_id 
#session_id=requests.Session()

driver = webdriver.Remote(command_executor=url,desired_capabilities={})
driver.close()   
driver.session_id = session_id

s.maximize_window()

/var/folders/j0/kbzw3v0x10927w9wwxmzx0jc0000gn/T/ipykernel_3055/2747771839.py:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  s = webdriver.Chrome( '/Users/hanshan/Downloads/chromedriver',options=op)


In [ ]:
from decimal import *
getcontext().prec = 4
def get_qqq_info():
    global s
    global url
    global session_id
    url = s.command_executor._url
    session_id = s.session_id 
    
    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id
    
    drop_down = s.find_element(By.ID, 'btnStrategy')
    drop_down.click()
    time.sleep(2)
    greeks = s.find_element(By.ID, 'greeks')
    greeks.click()
    time.sleep(2)
    
    refresh = s.find_element(By.ID, "refresh-all-quotes")
    refresh.click()
    
    time.sleep(5)
  
    last_trade_time = s.find_element(By.XPATH, '//*[@id="chains-symbol-entry"]/div[2]/div[2]/div[1]/div[3]')
    day1 = s.find_element(By.XPATH, '//*[@id="exp-0-header"]/tr[2]/td[1]/span')
    day2 = s.find_element(By.XPATH, '//*[@id="exp-1-header"]/tr[2]/td[1]/span')
    day3 = s.find_element(By.XPATH, '//*[@id="exp-2-header"]/tr[2]/td[1]/span')

    day_rows1 = s.find_element(By.XPATH, '//*[@id="exp-0-div"]/div[1]/span[2]/b[1]')

    day_rows2 = s.find_element(By.XPATH, '//*[@id="exp-1-div"]/div[1]/span[2]/b[1]')

    day_rows3 = s.find_element(By.XPATH, '//*[@id="exp-2-div"]/div[1]/span[2]/b[1]')
    
    
    elements = []
    elements = s.find_elements(By.CLASS_NAME, 'strike-row')
    print(elements)
    
    

    Time = []
    total_row_values = int(float(day_rows1.text)) + int(float(day_rows2.text)) + int(float(day_rows3.text))
    i = 0 
    while i != total_row_values:
        Time.append(last_trade_time.text[11:24])
        i = i + 1
    
    Day = []
    i = 0 
    while i != int(day_rows1.text):
        Day.append(day1.text[4:16])
        i = i +1
  
    i = 0 
    while i != int(day_rows2.text):
        Day.append(day2.text[4:16])
        i = i +1
    
    i = 0 
    while i != int(day_rows3.text):
        Day.append(day3.text[4:16])
        i = i +1



    Vega = []
    Theta = []
    Gamma = []
    Delta = []
    IV = []
    Bid = []
    Ask = []
    Strike = []
    Bid2 = []
    Ask2 = []
    IV2 = []
    Delta2 = []
    Gamma2  = []
    Theta2 = []
    Vega2 = []

    
    for element in elements:
        Vega.append(element.text.split(' ')[1])
        Theta.append(element.text.split(' ')[2])
        Gamma.append(element.text.split(' ')[3])
        Delta.append(element.text.split(' ')[4])
        IV.append(element.text.split(' ')[5])
        Bid.append(element.text.split(' ')[6])
        Ask.append(element.text.split(' ')[7])
        Strike.append(element.text.split(' ')[8])
        Bid2.append(element.text.split(' ')[9])
        Ask2.append(element.text.split(' ')[10])
        IV2.append(element.text.split(' ')[11])
        Delta2.append(element.text.split(' ')[12])
        Gamma2.append(element.text.split(' ')[13])
        Theta2.append(element.text.split(' ')[14])
        Vega2.append(element.text.split(' ')[15])

    url = s.command_executor._url
    session_id = s.session_id 

    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id

    d = {'Day': Day, 'Time': Time, 'Vega' : Vega, 'Theta' : Theta, 'Gamma' : Gamma, 'Delta' : Delta, "IV" : IV, "Bid" : Bid, "Ask" : Ask, "Strike" : Strike, "Bid2" : Bid2, "Ask2" : Ask2, 'IV2' : IV2, "Delta2" : Delta2, 'Gamma2' : Gamma2, "Theta2" : Theta2, 'Vega2' : Vega2}
    data_table_greeks = pd.DataFrame(data = d)
    #data_table_greeks
    data_table_greeks.to_sql('QQQ_Greeks', engine, if_exists = 'append', index=False)


    #Calls/Puts.
    url = s.command_executor._url
    session_id = s.session_id 
    
    driver = webdriver.Remote(command_executor=url,desired_capabilities={})
    driver.close()   
    driver.session_id = session_id
    time.sleep(2)
    drop_down = s.find_element(By.ID, 'btnStrategy')
    drop_down.click()
    time.sleep(2)
    calls_puts = s.find_element(By.ID, 'calls-and-puts')
    calls_puts.click()


    #time.sleep(5)
    #refresh = s.find_element(By.ID, "refresh-all-quotes")
    #refresh.click()

    
    
    
    time.sleep(6)
    last_trade_time = s.find_element(By.XPATH, '//*[@id="chains-symbol-entry"]/div[2]/div[2]/div[1]/div[3]')
    day1 = s.find_element(By.XPATH, '//*[@id="exp-0-header"]/tr[2]/td[1]/span')
    day2 = s.find_element(By.XPATH, '//*[@id="exp-1-header"]/tr[2]/td[1]')
    day3 = s.find_element(By.XPATH, '//*[@id="exp-2-header"]/tr[2]/td[1]/span')

    day_rows1 = s.find_element(By.XPATH, '//*[@id="exp-0-div"]/div[1]/span[2]/b[1]')

    day_rows2 = s.find_element(By.XPATH, '//*[@id="exp-1-div"]/div[1]/span[2]/b[1]')

    day_rows3 = s.find_element(By.XPATH, '//*[@id="exp-2-div"]/div[1]/span[2]/b[1]')
    
    elements2 = []
    elements2 = s.find_elements(By.CLASS_NAME, 'strike-row')

    
    Time = []
    total_row_values = int(float(day_rows1.text)) + int(float(day_rows2.text)) + int(float(day_rows3.text))
    i = 0 
    while i != total_row_values:
        Time.append(last_trade_time.text[11:24])
        i = i + 1
    
    Day = []
    i = 0 
    while i != int(day_rows1.text):
        Day.append(day1.text[4:16])
        i = i +1
  
    i = 0 
    while i != int(day_rows2.text):
        Day.append(day2.text[4:16])
        i = i +1
    
    i = 0 
    while i != int(day_rows3.text):
        Day.append(day3.text[4:16])
        i = i +1
    Volume = []
    OI = []
    Last = []
    Change = []
    Bid = []
    Ask = []
    Strike = []
    Bid2 = []
    Ask2 = []
    Last2 = []
    Change2 = []
    Volume2 = []
    OI2 = []
    

    for element in elements2:
        Volume.append(element.text.split(' ')[2])
        OI.append(element.text.split(' ')[3])
        Last.append(element.text.split(' ')[4])
        Change.append(element.text.split(' ')[5])
        Bid.append(element.text.split(' ')[6])
        Ask.append(element.text.split(' ')[7])
        Strike.append(element.text.split(' ')[8])
        Bid2.append(element.text.split(' ')[9])
        Ask2.append(element.text.split(' ')[10])
        Last2.append(element.text.split(' ')[11])
        Change2.append(element.text.split(' ')[12])
        Volume2.append(element.text.split(' ')[13])
        OI2.append(element.text.split(' ')[14])


    d2 = {'Day' : Day, 'Time' : Time, 'Volume' : Volume, 'OI' : OI, 'Last' : Last, 'Change' : Change, "Bid" : Bid, "Ask" : Ask, "Strike" : Strike, "Bid2" : Bid2, "Ask2" : Ask2, "Last2" : Last2, 'Change2' : Change2, "Volume2" : Volume2, 'OI2' : OI2}
    data_table_cp = pd.DataFrame(data = d2)
    #data_table_cp
    data_table_cp.to_sql('QQQ_cp', engine, if_exists = 'append', index=False)
    print('ran')
    
schedule.clear()
#                   *ALL TIME INTERVALS ARE WRITTEN IN MILITARY FORMAT*
#                            ENTER START VALUE HERE:
#                    Year  Month Date Hour Minute Second
pause.until(datetime(2022,   8,   15, 16 ,   35,    0   ))
get_qqq_info()
#                            ENTER END VALUE HERE:
#              run code until... (Time)
schedule.every(3.6).minutes.until("18:30").do(get_qqq_info)
while True:
    schedule.run_pending()
    time.sleep(1) 

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=102.0.5005.115)
Stacktrace:
0   chromedriver                        0x00000001005a1170 chromedriver + 4477296
1   chromedriver                        0x0000000100538c18 chromedriver + 4049944
2   chromedriver                        0x000000010018ad40 chromedriver + 191808
3   chromedriver                        0x000000010017785c chromedriver + 112732
4   chromedriver                        0x000000010017689c chromedriver + 108700
5   chromedriver                        0x0000000100176d38 chromedriver + 109880
6   chromedriver                        0x00000001001939f0 chromedriver + 227824
7   chromedriver                        0x000000010018d22c chromedriver + 201260
8   chromedriver                        0x000000010018cf58 chromedriver + 200536
9   chromedriver                        0x000000010018d5b0 chromedriver + 202160
10  chromedriver                        0x00000001001

In [1]:
import sqlite3 as sql
import os
import csv
#from sqlite3 import Error
import time

In [2]:
timestr = time.strftime("%Y_%m_%d")#"-%H%M%S")
conn=sql.connect('QQQ_information.db')
print("Exporting data into CSV............")
cursor = conn.cursor()
cursor.execute("select * from QQQ_cp")
with open("QQQ_cp_"+timestr+".csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter="\t")
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(cursor)

    dirpath = os.getcwd() + "/QQQ_cp.csv"
    print( "Data exported Successfully into {}".format(dirpath))

#except Error as e:
    #print(e)

# Close database connection
#finally:
    conn.close()

Exporting data into CSV............
Data exported Successfully into /Users/hanshan/Desktop/python project/QQQ_cp.csv


In [3]:
timestr = time.strftime("%Y_%m_%d")
conn=sql.connect('QQQ_information.db')
print("Exporting data into CSV............")
cursor = conn.cursor()
cursor.execute("select * from QQQ_Greeks")
with open("QQQ_Greeks_"+timestr+".csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter="\t")
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(cursor)

    dirpath = os.getcwd() + "/QQQ_cp.csv"
    print( "Data exported Successfully into {}".format(dirpath))

#except Error as e:
    #print(e)

# Close database connection
#finally:
    conn.close()

Exporting data into CSV............
Data exported Successfully into /Users/hanshan/Desktop/python project/QQQ_cp.csv


In [10]:
import pandas as pd
df1 = pd.read_csv(r'/Users/hanshan/Desktop/python project/QQQ_Greeks_2022_06_19.csv')
df1.info()
print(df1)

df1.to_excel(r'/Users/hanshan/Desktop/python project/QQQ_Greeks_2022_06_19.xlsx', index = False, header=True)

<class 'pandas.core.frame.DataFrame'>
Index: 6781 entries, Jun 17 to Jun 22
Data columns (total 1 columns):
 #   Column                                                                                      Non-Null Count  Dtype 
---  ------                                                                                      --------------  ----- 
 0   Day	Time	Vega	Theta	Gamma	Delta	IV	Bid	Ask	Strike	Bid2	Ask2	IV2	Delta2	Gamma2	Theta2	Vega2  6781 non-null   object
dtypes: object(1)
memory usage: 106.0+ KB
       Day\tTime\tVega\tTheta\tGamma\tDelta\tIV\tBid\tAsk\tStrike\tBid2\tAsk2\tIV2\tDelta2\tGamma2\tTheta2\tVega2
Jun 17   2022\t3:43:11 PM ET\t0.000\t-0.035\t0.000\t1....                                                        
Jun 17   2022\t3:43:11 PM ET\t0.000\t-0.035\t0.000\t1....                                                        
Jun 17   2022\t3:43:11 PM ET\t0.000\t-0.035\t0.000\t1....                                                        
Jun 17   2022\t3:43:11 PM ET\t0.00

ModuleNotFoundError: No module named 'openpyxl'